# NumPy & Pandas assignment helper

This notebook contains utilities to inspect the active source file, run unit tests programmatically, execute scripts, and capture outputs and tracebacks. It's oriented for use inside VS Code and follows the requested outline.

Instructions: Set the environment variable NOTEBOOK_ACTIVE_FILE to the path of the file you want to inspect, or set the variable `active_file_path` in the Setup cell.

In [ ]:
# Section 1: Setup — import libraries & configure paths
import os
import sys
import subprocess
import unittest
import io
import traceback
from datetime import datetime
from IPython.display import Markdown, HTML, display

# Active document path can be provided via env var or set here
active_file_path = os.environ.get('NOTEBOOK_ACTIVE_FILE', '')
repo_root = os.getcwd()

print(f"repo_root: {repo_root}")
print(f"active_file_path: {active_file_path}")

In [ ]:
# Section 2: Read the active document (open file)

def read_active_file(path, n_lines=30):
    if not path:
        display(Markdown('**No active file path set.** Set NOTEBOOK_ACTIVE_FILE env var or `active_file_path` variable.'))
        return None
    try:
        size = os.path.getsize(path)
        mtime = datetime.fromtimestamp(os.path.getmtime(path))
        with open(path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        display(Markdown(f"**File:** {path}  
**Size:** {size} bytes  
**Modified:** {mtime}"))
        preview = ''.join(lines[:n_lines])
        print(preview)
        return {'path': path, 'size': size, 'mtime': mtime, 'preview': preview}
    except Exception as e:
        display(Markdown('**Error reading file:**'))
        print(traceback.format_exc())
        return None

# Example usage (will show fallback message if not set)
_read_info = read_active_file(active_file_path)

In [ ]:
# Section 3 & 4: Run unit tests programmatically and capture output

def run_unittests(test_dir='tests'):
    loader = unittest.TestLoader()
    suite = loader.discover(start_dir=test_dir)
    buf = io.StringIO()
    runner = unittest.TextTestRunner(stream=buf, verbosity=2)
    result = runner.run(suite)
    output = buf.getvalue()
    # summary
    summary = {
        'testsRun': result.testsRun,
        'failures': len(result.failures),
        'errors': len(result.errors),
        'wasSuccessful': result.wasSuccessful(),
        'output': output,
    }
    return summary

# Example: run_unittests() will attempt to discover tests under ./tests
_test_summary = run_unittests()
print('Discovered tests run:', _test_summary['testsRun'])
print('Failures:', _test_summary['failures'])
print('Errors:', _test_summary['errors'])
print('
Captured output (truncated 200 chars):')
print(_test_summary['output'][:200])

In [ ]:
# Section 5 & 6: Execute scripts using subprocess and display outputs/tracebacks

def run_script(path, timeout=30):
    if not os.path.exists(path):
        return {'returncode': None, 'stdout': '', 'stderr': f'File not found: {path}'}
    start = datetime.now()
    try:
        completed = subprocess.run([sys.executable, path], capture_output=True, text=True, timeout=timeout)
        elapsed = (datetime.now() - start).total_seconds()
        return {
            'returncode': completed.returncode,
            'stdout': completed.stdout,
            'stderr': completed.stderr,
            'elapsed': elapsed,
        }
    except Exception as e:
        return {'returncode': None, 'stdout': '', 'stderr': traceback.format_exc()}

# Example: run the provided solution.py if present in repo root
_script_res = run_script(os.path.join(repo_root, 'solution.py'))
print('Return code:', _script_res['returncode'])
print('Stdout (first 400 chars):')
print(_script_res['stdout'][:400])
print('Stderr:')
print(_script_res['stderr'])

In [ ]:
# Section 7: Minimal example: function + unit tests (run & show results)

def add(a, b):
    return a + b

# Create tests programmatically
class TestAdd(unittest.TestCase):
    def test_add_positive(self):
        self.assertEqual(add(2, 3), 5)
    def test_add_negative(self):
        self.assertEqual(add(-1, -1), -2)

# Run tests programmatically
suite = unittest.TestLoader().loadTestsFromTestCase(TestAdd)
buf = io.StringIO()
res = unittest.TextTestRunner(stream=buf, verbosity=2).run(suite)
output = buf.getvalue()

# Parse and display
display(Markdown('### Minimal tests - results'))
print(output)

# Show parsed summary
display(Markdown(f"- testsRun: {res.testsRun}  \n- failures: {len(res.failures)}  \n- errors: {len(res.errors)}  \n- successful: {res.wasSuccessful()}"))

In [ ]:
# Section 8: Watch file changes and re-run tests (optional automation)

# NOTE: This is a simple polling loop example. For production use, prefer watchdog.
import time

def watch_and_run(path, interval=1.0, iterations=5):
    last_mtime = None
    for i in range(iterations):
        if not os.path.exists(path):
            display(Markdown(f"File not found: {path}"))
            return
        mtime = os.path.getmtime(path)
        if last_mtime is None:
            last_mtime = mtime
        if mtime != last_mtime:
            display(Markdown(f"Detected change in {path}, re-running tests"))
            _ = run_unittests()
            last_mtime = mtime
        time.sleep(interval)

# Usage example (disabled by default)
# watch_and_run(os.path.join(repo_root, 'solution.py'), interval=1.0, iterations=3)

display(Markdown('Section 8: Watcher example provided (not executed by default)'))